In [1]:
import os
from dotenv import load_dotenv
from nltk import toolbox
load_dotenv()
from IPython.display import Markdown, display

In [2]:
from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

/Users/nithurshen/SNU/MAT496/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nithurshen/SNU/MAT496/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import tool
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

@tool
def ddgs(source, query):
    """Integrates with Duck Duck Go Search engine, and fetches latest information from the web
    Allowed parameters for source: 'news', 'article', 'text', 'images', 'videos', 'answers'"""
    duck = DuckDuckGoSearchRun(source=source)
    return duck.invoke(query)
@tool
def python_interpreter(code):
    """A Python interpreter. Use this to execute Python code. Input should be a single string of valid Python code."""
    return PythonREPLTool(code)
@tool
def wikipedia(query):
    """A wrapper for the Wikipedia API. A useful tool for answering questions about general knowledge."""
    wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wiki.run(query)

tools = [ddgs, python_interpreter, wikipedia]

model_op = model.bind_tools(tools)

In [47]:
response = model_op.invoke("Execute a python code to create a variable x, and assign value 33, and multiply it by 5 and print it. Also take that number, and search the wikipedia site, if there are any athletes using that number, and check if the name matches with any current news")
response.tool_calls

[{'name': 'python_interpreter',
  'args': {'code': 'x = 33 * 5; print(x)'},
  'id': 'fy56bjmq4',
  'type': 'tool_call'},
 {'name': 'wikipedia',
  'args': {'query': 'athlete number 165'},
  'id': 'qd1p92vvd',
  'type': 'tool_call'},
 {'name': 'ddgs',
  'args': {'query': 'athlete number 165 news', 'source': 'news'},
  'id': '1grcmv8dz',
  'type': 'tool_call'}]